In [193]:
import numpy as np

# Load the matrix from a .npy file
time_series = np.load('dataset.npy')

# Normalize
time_series = time_series / 10 * 1


def map_back(array):
    return array / (2*np.pi) * 10

In [194]:

print(time_series.shape)

(6000, 20, 10)


In [206]:

from qiskit import QuantumCircuit, transpile, Aer, IBMQ, execute
from qiskit.result import marginal_counts

depth = 9
q_wires = 9

def vqc_unit(params,qc,thetas):
    # Encoding
    for theta_i in thetas:
        qc.u(theta_i,0,0,1)
    
    # qc.barrier()
    # Interaction
    for m in range(depth + 1):
        for i in range(q_wires):
            qc.u(params[m, i, 0], params[m, i, 1], params[m, i, 2],i)
            
        #if m != depth:
        #    qc.cx(1,0)

backend = Aer.get_backend('aer_simulator')

def characteristic_results_from_outcome(counts,shots):
        
        """ 
        based on the outcome of an experiment, this function returns the distribution of
        each characteristic for each finch.
        counts (dict): outcome of experiment/simulation
        shots (int): number of repeats of the experiment/simulation
        """

        characteristic_results = np.zeros(9)

        for kk in range(len(characteristic_results)):
            for ii in range(2**(len(characteristic_results)-1)):
                string = np.binary_repr(ii,width=(len(characteristic_results)-1))[:kk]+str(1)+np.binary_repr(ii,width=(len(characteristic_results)-1))[kk:]
                if string in counts:
                    characteristic_results[kk] += counts[string]

        return characteristic_results/shots

# it's a multi-to-one Quantum RNN
def circuit_vqc(weights, theta):
    # input a time series
    
    circuit = QuantumCircuit(q_wires, q_wires)
    
    for i in range(len(theta)-1):
        vqc_unit(weights,circuit,theta[i])
        circuit.reset([1])

    vqc_unit(weights,circuit,theta[-1])   
    
    #circuit.measure([1], [0])
    
    circuit.measure(range(9), range(9))
    
    count = execute(circuit, backend=backend,shots=n_shots).result().get_counts()
    characteristic_results = characteristic_results_from_outcome(count, n_shots)
    theta_i = np.arcsin(np.sqrt(characteristic_results)) / (2*np.pi)
    return theta_i #np.flip(2*theta_i)
    
    '''
    num_ones = list()
    for i in range(9):
        simp_counts0 = marginal_counts(run_result, indices=[i]).get_counts()
    
        num_ones.append(simp_counts0['1'] if '1' in simp_counts0 else 0)
    
    print(num_ones)
    
    num_ones = np.array(num_ones)
    #print(circuit.draw())
    
    return num_ones/n_shots * 2*np.pi #np.tan((num_ones/n_shots - .5) * 2.75)
    '''

In [207]:
train_costs = list()
def square_loss(labels, predictions):
    loss = []
    for l, p in zip(labels, predictions):
        #print("l: ", l)
        #print("p: ", p)
        #print("the len: ", len(l))
        #print((l - p) ** 2)
        loss.append((l - p) ** 2)
    #print(np.array(loss).shape)
    loss = np.mean(loss)
    #print(loss)
    return loss

In [208]:
def cost_spsa(weights, seed=0):
    global train_costs, val_costs, best_val, best_val_w
    
    val_freq = 10
    
    input_len = 6
    
    np.random.seed(seed)
    batch_inds = np.random.randint(time_series.shape[0], size=batch_size)
    
    # X contains a time series of every patients
    X = time_series[batch_inds, :input_len, :9]
    
    weights = weights.reshape(weights_init_qaoa.shape)
    
    predictions = [circuit_vqc(weights, x) for x in X] 
    
    loss = square_loss(time_series[batch_inds, input_len, :9], predictions)
    
    train_costs.append(loss)
    
    '''
    if len(train_costs) % val_freq == 0:
      val_loss = val_cost(cur_weights)
      val_costs.append(val_loss)
      print(
          f"Iteration = {len(train_costs)}, "
          f"Val Loss = {val_loss} "
          f"Train Loss = {np.mean(train_costs[-val_freq:])}"
      )
      if val_loss < best_val:
        best_val_w = cur_weights.copy()
        best_val = val_loss
        print("Update best val w")
    '''
    print(loss)
    return loss

def val_cost(weights, val_angles, val_angles_y):
  weights = weights.reshape(weights_init_qaoa.shape)
  predictions = [circuit_vqc(weights, x) for x in val_angles] 
  return square_loss(val_angles_y[:, -1], predictions)

In [209]:
#!pip install noisyopt

import time
from noisyopt import minimizeSPSA

weights_init_qaoa = 2 * np.pi * np.random.uniform(-1, 1, size=(depth + 1, q_wires, 3))
# weights_4_val = np.array([ 13.26030476,  -4.85954186,  -1.48417534,  -0.57736945,
#          4.6643797 ,  -1.18933862,   4.95008181,  13.32304439,
#         -4.64256665,  -2.89863502,   4.90275961, -13.28259991])
# weights_init_qaoa = np.array(weights_4_val)

cost_spsa(weights_init_qaoa, 0)

0.12527832504154224


0.12527832504154224

In [ ]:

train_costs = []
val_costs = []
cur_weights = weights_init_qaoa.flatten().copy()

batch_size = 64
n_shots = 50
st = time.time()
step_length = .01
pair_distance = 1 / np.sqrt(n_shots)
res = minimizeSPSA(
    cost_spsa,
    x0=cur_weights.copy(),
    niter=200,
    paired=True,
    c=pair_distance,
    a=step_length,
)
cur_weights = res.x
end = time.time()
print(f"Seconds: {end - st}")

0.11655200106526947
0.11021877582466431
0.12736233086192147
0.13376711553964243
0.13903753426014365
0.14131778047092483
0.12290943265966568
0.116739643933681
0.12901348380824038
0.13309158469232063
0.13319562060247792
0.1271468732630422
0.11717401064713551
0.11885509483085276
0.12846739580571004
0.13692184011047429
0.11273344907551719
0.11100503432077483
0.1130520115847408
0.0965989213857294
0.14365553590498098
0.14552000498744264
0.1028409244496936
0.1248724732111364
0.1015658218090314
0.09550214822056081
0.14178659219438228
0.15307210009073927
0.09903101217317771
0.10487414289267558
0.13496246791735622
0.11104045979361626
0.12146540201325585
0.10246036647316333
0.13035869808859868
0.12628659612271492
0.10800496374915111
0.11257136042756614
0.11495539603074582
0.12846833931601187
0.12828798323120053
0.13753443955263636
0.106764802667444
0.14081299030446098
0.09558176792535122
0.08508244034772251
0.14008712447872312
0.14994708119779182
0.11611413770723071
0.11791123697626338
0.10543495

In [ ]:
import matplotlib.pyplot as plt


# Plot the loss function
plt.plot(range(len(train_costs)), train_costs)